In [1]:
import os, numpy as np, sys, matplotlib as mpl, SimpleITK as sitk
import pandas as pd, matplotlib.pyplot as plt
sys.path.append("/jukebox/wang/ahoag/brainpipe")
from skimage.external import tifffile
from collections import Counter
from tools.imageprocessing.orientation import fix_orientation
from tools.registration.transform import count_structure_lister, transformed_pnts_to_allen_helper_func
from tools.registration.register import transformix_command_line_call
from tools.utils.io import makedir

import matplotlib.pyplot as plt
%matplotlib inline

## Run transformix on Jess' brains if it hasn't been run yet. 

This step uses the 488 -> atlas registration parameters that we got from running brainpipe to the injection site 647 volume. Since 488 and 647 are already aligned this 

In [5]:
n_brains = 20
brains = [f"092220_vectorandcrusIbil_openfieldonly-{str(x).zfill(3)}" for x in range(1,n_brains+1)]
brains

['092220_vectorandcrusIbil_openfieldonly-001',
 '092220_vectorandcrusIbil_openfieldonly-002',
 '092220_vectorandcrusIbil_openfieldonly-003',
 '092220_vectorandcrusIbil_openfieldonly-004',
 '092220_vectorandcrusIbil_openfieldonly-005',
 '092220_vectorandcrusIbil_openfieldonly-006',
 '092220_vectorandcrusIbil_openfieldonly-007',
 '092220_vectorandcrusIbil_openfieldonly-008',
 '092220_vectorandcrusIbil_openfieldonly-009',
 '092220_vectorandcrusIbil_openfieldonly-010',
 '092220_vectorandcrusIbil_openfieldonly-011',
 '092220_vectorandcrusIbil_openfieldonly-012',
 '092220_vectorandcrusIbil_openfieldonly-013',
 '092220_vectorandcrusIbil_openfieldonly-014',
 '092220_vectorandcrusIbil_openfieldonly-015',
 '092220_vectorandcrusIbil_openfieldonly-016',
 '092220_vectorandcrusIbil_openfieldonly-017',
 '092220_vectorandcrusIbil_openfieldonly-018',
 '092220_vectorandcrusIbil_openfieldonly-019',
 '092220_vectorandcrusIbil_openfieldonly-020']

In [6]:
for brain in brains:
    print(brain)
    src =  os.path.join('/jukebox/LightSheetData/lightserv/jverpeut',
        '092220_vectorandcrusIbil_openfieldonly',brain,
        'imaging_request_1/output/processing_request_1/resolution_1.1x')
    mv = os.path.join(src,f"{brain}_resized_ch01_resampledforelastix.tif")
    out =  os.path.join(src, "injection","transformix_injection_to_pma")
    if not os.path.exists(out):
        os.mkdir(out)
    resultfile = os.path.join(out,'result.tif')
    if os.path.exists(resultfile):
        print("transformed injection file for already exists")
        continue
    transformfile = os.path.join(src,"elastix","TransformParameters.1.txt")    
    assert os.path.exists(transformfile)
    print("Calling transformix")
    transformix_command_line_call(mv, out, transformfile)

092220_vectorandcrusIbil_openfieldonly-001
transformed injection file for already exists
092220_vectorandcrusIbil_openfieldonly-002
transformed injection file for already exists
092220_vectorandcrusIbil_openfieldonly-003
transformed injection file for already exists
092220_vectorandcrusIbil_openfieldonly-004
transformed injection file for already exists
092220_vectorandcrusIbil_openfieldonly-005
transformed injection file for already exists
092220_vectorandcrusIbil_openfieldonly-006
transformed injection file for already exists
092220_vectorandcrusIbil_openfieldonly-007
transformed injection file for already exists
092220_vectorandcrusIbil_openfieldonly-008
transformed injection file for already exists
092220_vectorandcrusIbil_openfieldonly-009
transformed injection file for already exists
092220_vectorandcrusIbil_openfieldonly-010
transformed injection file for already exists
092220_vectorandcrusIbil_openfieldonly-011
transformed injection file for already exists
092220_vectorandcrusI

In [7]:
def find_site(im, thresh=10, filter_kernel=(5,5,5), num_sites_to_keep=1):
    """Find a connected area of high intensity, using a basic filter + threshold + connected components approach
    
    by: bdeverett
    Parameters
    ----------
    img : np.ndarray
        3D stack in which to find site (technically need not be 3D, so long as filter parameter is adjusted accordingly)
    thresh: float
        threshold for site-of-interest intensity, in number of standard deviations above the mean
    filter_kernel: tuple
        kernel for filtering of image before thresholding
    num_sites_to_keep: int, number of injection sites to keep, useful if multiple distinct sites
    
    Returns
    --------
    bool array of volume where coordinates where detected
    """
    from scipy.ndimage.filters import gaussian_filter as gfilt
    from scipy.ndimage import label
    if type(im) == str: im = tifffile.imread(im)

    filtered = gfilt(im, filter_kernel)
    thresholded = filtered > filtered.mean() + thresh*filtered.std() 
    labelled,nlab = label(thresholded)

    if nlab == 0:
        raise Exception("Site not detected, try a lower threshold?")
    elif nlab == 1:
        return labelled.astype(bool)
    elif num_sites_to_keep == 1:
        sizes = [np.sum(labelled==i) for i in range(1,nlab+1)]
        return labelled == np.argmax(sizes)+1
    else:
        sizes = [np.sum(labelled==i) for i in range(1,nlab+1)]
        vals = [i+1 for i in np.argsort(sizes)[-num_sites_to_keep:][::-1]]
        return np.in1d(labelled, vals).reshape(labelled.shape)

In [8]:
pma_ann_file = "/jukebox/LightSheetTransfer/atlas/annotation_sagittal_atlas_20um_iso.tif"
pma_ann = tifffile.imread(pma_ann_file)
pma_tiss_file = "/jukebox/LightSheetTransfer/atlas/sagittal_atlas_20um_iso.tif"
pma_tiss = tifffile.imread(pma_tiss_file)

In [9]:
pma_ann_cropped = pma_ann[:, 450:, :] # cropped to focus on cerebellum only
pma_tiss_cropped = pma_tiss[:, 450:, :] # cropped to focus on cerebellum only

In [20]:
#ch{eck if reorientation is necessary
#    src = "/home/wanglab/mounts/wang/Jess/lightsheet_output/201904_ymaze_cfos/injection/processed/an26/elastix/an26_ymazefos_021519_1d3x_647_008na_1hfds_z10um_400msec_ch00/result.1.tif"
#    src = orientation_crop_check(src, axes = ("2","1","0"), crop = "[:,500:,:200]")
#    
#    #optimize detection parameters for inj det
#    optimize_inj_detect(src, threshold=5, filter_kernel = (1,1,1))
#    
#run
#suggestion: save_individual=True,
#then inspect individual brains, which you can then remove bad brains from list and rerun function
kwargs = { 
      "filter_kernel": (5, 5, 5),
      "threshold": 3,
      "num_sites_to_keep": 1,
      "injectionscale": 45000, 
      "imagescale": 3,
      "reorientation": ("2","0","1"),
      "crop": "[:, 450:, :]",
      "crop_atlas": "[:, 450:, :]",
      "save_individual": True, 
      "save_tif": True,
      "colormap": "jet", 
      "atlas": "/jukebox/LightSheetTransfer/atlas/sagittal_atlas_20um_iso.tif",
      "annotation":"/jukebox/LightSheetTransfer/atlas/annotation_sagittal_atlas_20um_iso.tif",
      "id_table": "/jukebox/LightSheetTransfer/atlas/ls_id_table_w_voxelcounts.xlsx"
    }    
nonzeros = []

for ii,brain in enumerate(brains):
    print(brain)
    pooled_dst = os.path.join("/jukebox/LightSheetData/lightserv/jverpeut",
        "092220_vectorandcrusIbil_openfieldonly","injection","pooled_analysis")
    
    src =  os.path.join('/jukebox/LightSheetData/lightserv/jverpeut',
        '092220_vectorandcrusIbil_openfieldonly',brain,
        'imaging_request_1/output/processing_request_1/resolution_1.1x')
    inj_pth =  os.path.join(src, "injection","transformix_injection_to_pma")
    inputlist = []
    inputlist.append(os.path.join(inj_pth, "result.tif"))
    
    print()

    injscale = kwargs["injectionscale"] if "injectionscale" in kwargs else 1
    imagescale = kwargs["imagescale"] if "imagescale" in kwargs else 1
    axes = kwargs["reorientation"] if "reorientation" in kwargs else ("0","1","2")
    cmap = kwargs["colormap"] if "colormap" in kwargs else "plasma"
    id_table = kwargs["id_table"] if "id_table" in kwargs else "/jukebox/LightSheetTransfer/atlas/allen_atlas/allen_id_table.xlsx"
    save_tif = kwargs["save_tif"] if "save_tif" in kwargs else False
    save_individual = kwargs["save_individual"] 
    num_sites_to_keep = kwargs["num_sites_to_keep"] if "num_sites_to_keep" in kwargs else 1
    ann = sitk.GetArrayFromImage(sitk.ReadImage(kwargs["annotation"]))
    if kwargs["crop"]: ann = eval("ann{}".format(kwargs["crop"]))   
    allen_id_table=pd.read_excel(id_table)
    
    impth = inputlist[0]
    
    print("loading image: ", impth)
    print()
    im = tifffile.imread(impth)

    if kwargs["crop"]: im = eval("im{}".format(kwargs["crop"]))#; print im.shape

   
    savename_tif = os.path.join(pooled_dst,"{}".format(brain)+"_inj.tif")
    if os.path.exists(savename_tif):
        print("loading injection site array from file")
        arr = tifffile.imread(savename_tif)
    else:
        print("Finding injection site")
        print()
        #segment
        arr = find_site(im, thresh=kwargs["threshold"], filter_kernel=kwargs["filter_kernel"], 
                    num_sites_to_keep=num_sites_to_keep)*injscale
        if save_tif:
            tifffile.imsave(savename_tif, arr.astype("float32"))
            print(f"Saved {savename_tif}")
            print()

    if save_individual:
        savename_individual = os.path.join(pooled_dst,"{}".format(brain)+".png")
        im_scaled = im*imagescale
        im_maxproj = np.max(im_scaled, axis=0)
        arr_maxproj = np.max(arr, axis=0)
        pma_tiss_cropped_maxproj = np.max(pma_tiss_cropped, axis=0)

        im_maxproj_coronal = np.fliplr(np.rot90(np.max(fix_orientation(im_scaled, axes=axes), axis=0),k=3))
        arr_maxproj_coronal = np.fliplr(np.rot90(np.max(fix_orientation(arr, axes=axes), axis=0),k=3))
        atl_maxproj_coronal = np.fliplr(np.rot90(np.max(fix_orientation(pma_tiss_cropped, axes=axes), axis=0),k=3))

        # b = np.concatenate((np.fliplr(np.rot90(np.max(fix_orientation(im_scaled, axes=axes), axis=0),k=3)), 
        #                     np.fliplr(np.rot90(np.max(fix_orientation(arr.astype("uint16"), axes=axes), axis=0),k=3))),
        #                    axis=1)
        cmap='jet'
        fontsize=22
        fig =plt.figure(figsize=(12,8))
        ax_im_coronal=fig.add_subplot(231)
        ax_im_coronal.imshow(im_maxproj_coronal,vmin=0,vmax=1500, cmap=cmap, alpha=1)
        # ax_im_coronal.axis('off')
        ax_im_coronal.set_title("Injection channel",fontsize=fontsize, pad=10)
        ax_im_coronal.set_ylabel("Coronal",fontsize=fontsize,labelpad=12)
        ax_im_coronal.set_xticks([])
        ax_im_coronal.set_yticks([])

        ax_arr_coronal=fig.add_subplot(232)
        ax_arr_coronal.imshow(arr_maxproj_coronal,vmin=0,vmax=1500, cmap='gray', alpha=1)
        ax_arr_coronal.axis('off')
        ax_arr_coronal.set_title("Segmentation mask",fontsize=fontsize, pad=10)

        ax_atl_coronal=fig.add_subplot(233)
        ax_atl_coronal.imshow(atl_maxproj_coronal+arr_maxproj_coronal,vmin=0,vmax=300, cmap='gray', alpha=1)
        ax_atl_coronal.axis('off')
        ax_atl_coronal.set_title("Princeton Mouse Atlas\n+mask",fontsize=fontsize, pad=10)


        ax_im=fig.add_subplot(234)
        ax_im.imshow(im_maxproj,vmin=0,vmax=1500, cmap=cmap, alpha=1)
        ax_im.set_ylabel("Sagittal",fontsize=fontsize,labelpad=12)
        ax_im.set_xticks([])
        ax_im.set_yticks([])

        ax_inj=fig.add_subplot(235)
        ax_inj.imshow(arr_maxproj,vmin=0,vmax=1500, cmap='gray', alpha=1)
        ax_inj.axis('off')


        ax_atl=fig.add_subplot(236)
        ax_atl.imshow(pma_tiss_cropped_maxproj+arr_maxproj,vmin=0,vmax=300, cmap='gray', alpha=1)
        ax_atl.axis('off')

        plt.tight_layout()
        plt.savefig(savename_individual, dpi=300, transparent=True)
        print(f"saved {savename_individual}")
        plt.close()
        print()
        
    #cell counts to csv
    print("   finding nonzero pixels for voxel counts...")      
    nz = np.nonzero(arr)
    nonzeros.append(list(zip(*nz))) #<-for pooled image
    pos = transformed_pnts_to_allen_helper_func(np.asarray(list(zip(*[nz[2], nz[1], nz[0]]))), ann)
    tdf = count_structure_lister(allen_id_table, *pos)
    if ii == 0: 
        df = tdf.copy()
        countcol = "count" if "count" in df.columns else "cell_count"
        df.drop([countcol], axis=1, inplace=True)

    df[brain] = tdf[countcol]
    

df.to_csv(os.path.join(pooled_dst,"voxel_counts.csv"))
print("\n\nCSV file of cell counts, saved as {}\n\n\n".format(os.path.join(pooled_dst,"voxel_counts.csv")))  

#load atlas and generate final figure
print("Generating final figure...")      
atlas = tifffile.imread(kwargs["atlas"])
#cropping
#if "crop_atlas" not in kwargs:
if kwargs["crop_atlas"]: atlas = eval("atlas{}".format(kwargs["crop_atlas"]))

#condense nonzero pixels
nzs = [str(x) for xx in nonzeros for x in xx] #this list has duplicates if two brains had the same voxel w label
c = Counter(nzs)
array = np.zeros_like(atlas)
print("Collecting nonzero pixels for pooled image...")
tick = 0
#generating pooled array where voxel value = total number of brains with that voxel as positive
for k,v in c.items():
    k = [int(xx) for xx in k.replace("(","").replace(")","").split(",")]
    array[k[0], k[1], k[2]] = int(v)
    tick+=1
    if tick % 50000 == 0: print("   {}".format(tick))

#reslice
atlas = fix_orientation(atlas, axes = axes)
arr = fix_orientation(array, axes = axes)

my_cmap = eval("plt.cm.{}(np.arange(plt.cm.RdBu.N))".format(cmap))
my_cmap[:1,:4] = 0.0  
my_cmap = mpl.colors.ListedColormap(my_cmap)
my_cmap.set_under("w")
plt.figure()
plt.imshow(np.max(atlas, axis=0), cmap="gray")
plt.imshow(np.max(arr, axis=0), alpha=0.99, cmap=my_cmap); plt.colorbar(); plt.axis("off")
plt.savefig(os.path.join(pooled_dst,"heatmap.pdf"), dpi = 300, transparent=True)
plt.close()

print("Saved as {}".format(os.path.join(pooled_dst,"heatmap.pdf")))  


092220_vectorandcrusIbil_openfieldonly-001

loading image:  /jukebox/LightSheetData/lightserv/jverpeut/092220_vectorandcrusIbil_openfieldonly/092220_vectorandcrusIbil_openfieldonly-001/imaging_request_1/output/processing_request_1/resolution_1.1x/injection/transformix_injection_to_pma/result.tif

loading injection site array from file
saved /jukebox/LightSheetData/lightserv/jverpeut/092220_vectorandcrusIbil_openfieldonly/injection/pooled_analysis/092220_vectorandcrusIbil_openfieldonly-001.png

   finding nonzero pixels for voxel counts...

*************0 points do not map to atlas*********
092220_vectorandcrusIbil_openfieldonly-002

loading image:  /jukebox/LightSheetData/lightserv/jverpeut/092220_vectorandcrusIbil_openfieldonly/092220_vectorandcrusIbil_openfieldonly-002/imaging_request_1/output/processing_request_1/resolution_1.1x/injection/transformix_injection_to_pma/result.tif

loading injection site array from file
saved /jukebox/LightSheetData/lightserv/jverpeut/092220_vectorandc

092220_vectorandcrusIbil_openfieldonly-014

loading image:  /jukebox/LightSheetData/lightserv/jverpeut/092220_vectorandcrusIbil_openfieldonly/092220_vectorandcrusIbil_openfieldonly-014/imaging_request_1/output/processing_request_1/resolution_1.1x/injection/transformix_injection_to_pma/result.tif

Finding injection site

Saved /jukebox/LightSheetData/lightserv/jverpeut/092220_vectorandcrusIbil_openfieldonly/injection/pooled_analysis/092220_vectorandcrusIbil_openfieldonly-014_inj.tif

saved /jukebox/LightSheetData/lightserv/jverpeut/092220_vectorandcrusIbil_openfieldonly/injection/pooled_analysis/092220_vectorandcrusIbil_openfieldonly-014.png

   finding nonzero pixels for voxel counts...

*************0 points do not map to atlas*********
092220_vectorandcrusIbil_openfieldonly-015

loading image:  /jukebox/LightSheetData/lightserv/jverpeut/092220_vectorandcrusIbil_openfieldonly/092220_vectorandcrusIbil_openfieldonly-015/imaging_request_1/output/processing_request_1/resolution_1.1x/injec

In [4]:
merged_csv = '/jukebox/LightSheetData/lightserv/jverpeut/092220_vectorandcrusIbil_openfieldonly/injection/pooled_analysis/voxel_counts.csv'

In [5]:
df = pd.read_csv(merged_csv)

In [6]:
df

,Unnamed: 0,Unnamed: 0.1,name,acronym,id,atlas_id,parent_structure_id,parent_name,parent_acronym,voxels_in_structure,...,092220_vectorandcrusIbil_openfieldonly-011,092220_vectorandcrusIbil_openfieldonly-012,092220_vectorandcrusIbil_openfieldonly-013,092220_vectorandcrusIbil_openfieldonly-014,092220_vectorandcrusIbil_openfieldonly-015,092220_vectorandcrusIbil_openfieldonly-016,092220_vectorandcrusIbil_openfieldonly-017,092220_vectorandcrusIbil_openfieldonly-018,092220_vectorandcrusIbil_openfieldonly-019,092220_vectorandcrusIbil_openfieldonly-020
0,0,0,root,root,997,-1,NaN,NaN,NaN,87,...,0,0,0,0,0,0,0,0,0,0
1,1,1,Basic cell groups and regions,grey,8,0,997.0,root,root,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2,Cerebrum,CH,567,70,8.0,Basic cell groups and regions,grey,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3,Cerebral cortex,CTX,688,85,567.0,Cerebrum,CH,0,...,0,0,0,0,0,0,0,0,0,0
4,4,4,Cortical plate,CTXpl,695,86,688.0,Cerebral cortex,CTX,0,...,0,0,0,0,0,0,0,0,0,0
5,5,5,Isocortex,Isocortex,315,746,695.0,Cortical plate,CTXpl,0,...,0,0,0,0,0,0,0,0,0,0
6,6,6,"Frontal pole, cerebral cortex",FRP,184,871,315.0,Isocortex,Isocortex,0,...,0,0,0,0,0,0,0,0,0,0
7,7,7,"Frontal pole, layer 1",FRP1,68,998,184.0,"Frontal pole, cerebral cortex",FRP,28030,...,0,0,0,0,0,0,0,0,0,0
8,8,8,"Frontal pole, layer 2/3",FRP2/3,667,1073,184.0,"Frontal pole, cerebral cortex",FRP,24592,...,0,0,0,0,0,0,0,0,0,0
9,9,9,"Frontal pole, layer 5",FRP5,526157192,NaN,184.0,"Frontal pole, cerebral cortex",FRP,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
# Get rid of first two unncessary columns
df.drop(columns = ['Unnamed: 0','Unnamed: 0.1'],inplace=True)

In [40]:
df

,name,acronym,id,atlas_id,parent_structure_id,parent_name,parent_acronym,voxels_in_structure,092220_vectorandcrusIbil_openfieldonly-001,092220_vectorandcrusIbil_openfieldonly-002,...,092220_vectorandcrusIbil_openfieldonly-011,092220_vectorandcrusIbil_openfieldonly-012,092220_vectorandcrusIbil_openfieldonly-013,092220_vectorandcrusIbil_openfieldonly-014,092220_vectorandcrusIbil_openfieldonly-015,092220_vectorandcrusIbil_openfieldonly-016,092220_vectorandcrusIbil_openfieldonly-017,092220_vectorandcrusIbil_openfieldonly-018,092220_vectorandcrusIbil_openfieldonly-019,092220_vectorandcrusIbil_openfieldonly-020
0,root,root,997,-1,NaN,NaN,NaN,87,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Basic cell groups and regions,grey,8,0,997.0,root,root,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cerebrum,CH,567,70,8.0,Basic cell groups and regions,grey,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Cerebral cortex,CTX,688,85,567.0,Cerebrum,CH,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cortical plate,CTXpl,695,86,688.0,Cerebral cortex,CTX,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Isocortex,Isocortex,315,746,695.0,Cortical plate,CTXpl,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Frontal pole, cerebral cortex",FRP,184,871,315.0,Isocortex,Isocortex,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Frontal pole, layer 1",FRP1,68,998,184.0,"Frontal pole, cerebral cortex",FRP,28030,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Frontal pole, layer 2/3",FRP2/3,667,1073,184.0,"Frontal pole, cerebral cortex",FRP,24592,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Frontal pole, layer 5",FRP5,526157192,NaN,184.0,"Frontal pole, cerebral cortex",FRP,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df.columns[10:]

Index(['092220_vectorandcrusIbil_openfieldonly-001',
       '092220_vectorandcrusIbil_openfieldonly-002',
       '092220_vectorandcrusIbil_openfieldonly-003',
       '092220_vectorandcrusIbil_openfieldonly-004',
       '092220_vectorandcrusIbil_openfieldonly-005',
       '092220_vectorandcrusIbil_openfieldonly-006',
       '092220_vectorandcrusIbil_openfieldonly-007',
       '092220_vectorandcrusIbil_openfieldonly-008',
       '092220_vectorandcrusIbil_openfieldonly-009',
       '092220_vectorandcrusIbil_openfieldonly-010',
       '092220_vectorandcrusIbil_openfieldonly-011',
       '092220_vectorandcrusIbil_openfieldonly-012',
       '092220_vectorandcrusIbil_openfieldonly-013',
       '092220_vectorandcrusIbil_openfieldonly-014',
       '092220_vectorandcrusIbil_openfieldonly-015',
       '092220_vectorandcrusIbil_openfieldonly-016',
       '092220_vectorandcrusIbil_openfieldonly-017',
       '092220_vectorandcrusIbil_openfieldonly-018',
       '092220_vectorandcrusIbil_openfieldonly

In [42]:
rows_with_nonzero_counts = animals_df[(animals_df.select_dtypes(include=['number']) != 0).any(1)].index

In [48]:
df.iloc[rows_with_nonzero_counts]

,name,acronym,id,atlas_id,parent_structure_id,parent_name,parent_acronym,voxels_in_structure,092220_vectorandcrusIbil_openfieldonly-001,092220_vectorandcrusIbil_openfieldonly-002,...,092220_vectorandcrusIbil_openfieldonly-011,092220_vectorandcrusIbil_openfieldonly-012,092220_vectorandcrusIbil_openfieldonly-013,092220_vectorandcrusIbil_openfieldonly-014,092220_vectorandcrusIbil_openfieldonly-015,092220_vectorandcrusIbil_openfieldonly-016,092220_vectorandcrusIbil_openfieldonly-017,092220_vectorandcrusIbil_openfieldonly-018,092220_vectorandcrusIbil_openfieldonly-019,092220_vectorandcrusIbil_openfieldonly-020
976,Infracerebellar nucleus,ICB,372,470,370.0,"Medulla, motor related",MY-mot,4551,0,0,...,0,0,185,0,0,0,0,0,0,0
1014,Cerebellum,CB,512,63,8.0,Basic cell groups and regions,grey,70221,0,0,...,0,207,1270,375,58,34,42,12,0,94
1046,"Declive (VI), subdivision A",DEC6a,936,399a,1138.0,Declive (VI),DEC,234090,0,11,...,174,210,11,19604,0,330,663,368,0,0
1047,"Declive (VI), subdivision B",DEC6b,1134,399b,1138.0,Declive (VI),DEC,48881,0,319,...,0,3319,1372,20604,0,1471,4184,5004,0,3085
1051,Folium-tuber vermis (VII),FOTU,944,400,645.0,Vermal regions,VERM,110362,0,880,...,0,73899,20036,59025,49,77719,34451,77766,0,84766
1055,Pyramus (VIII),PYR,951,401,645.0,Vermal regions,VERM,135220,0,0,...,0,17008,789,18952,0,33137,56,8571,0,23982
1059,Uvula (IX),UVU,957,402,645.0,Vermal regions,VERM,275015,0,0,...,0,0,0,0,2,500,0,461,0,0
1063,Nodulus (X),NOD,968,403,645.0,Vermal regions,VERM,146640,0,0,...,0,0,0,0,0,193,0,137,0,0
1068,Simplex lobule,SIM,1007,408,1073.0,Hemispheric regions,HEM,517570,0,0,...,6,0,0,806,0,0,0,0,0,0
1073,Crus 1,ANcr1,1056,414,1017.0,Ansiform lobule,AN,516817,177,14,...,26412,1085,3,7232,179,0,1156,0,448,0


In [50]:
df.to_csv(merged_csv) # overwrites file (just removes first two bad columns)